# PAISE 2025 Plots

In [ ]:
# Common imports.
from pathlib import Path
from collections import defaultdict

%matplotlib widget
import base

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

import dfaas_env
import dfaas_utils
import dfaas_upperbound

## Experiment loading

In [ ]:
prefix_dir = Path("../results/paise2025")

exps_dir = []
for exp in prefix_dir.iterdir():
    exps_dir.append(exp)

assert len(exps_dir) > 0, "must select at least one experiment"

# Preload the data (result.json file) for all selected experiments.
raw_exp_data = {}
for exp_dir in exps_dir:
    raw_exp_data[exp_dir] = dfaas_utils.parse_result_file(exp_dir / "result.json")

# Create the reference environment based on DFaaS.
for exp_dir in exps_dir:
    env = base.get_env(exp_dir)
    if env.__class__ == dfaas_env.DFaaS:
        break

# At least one experiment must be of type DFaaS. SingleDFaaS is used only as
# reference.
assert env.__class__ == dfaas_env.DFaaS, f"{env.__class__}"

print("Selected experiments:")
for exp_dir in exps_dir:
    print(f"  - {exp_dir.name}")

## Enqueued requests

In [ ]:
# Common functions for processed requests.


def get_enqueued_requests_data_upperbound(iters):
    iterations = len(iters)
    final_data = {}
    final_data["input_reqs_avg"] = np.empty(iterations)
    final_data["enqueued_reqs_local_avg"] = np.empty(iterations)
    final_data["enqueued_reqs_forward_avg"] = np.empty(iterations)

    # For each iteration, calculate the metrics for each episode played,
    # then average the values for the number of episodes of that iteration.
    for iter_idx in range(iterations):
        episodes = iters[iter_idx]["env_runners"]["episodes_this_iter"]
        iter_data = iters[iter_idx]["env_runners"]["hist_stats"]

        # Create the data dictionary that contains the metrics for each
        # episode in this iteration.
        data = {}
        data["input_reqs"] = np.empty(episodes, dtype=np.int32)
        data["enqueued_reqs_local"] = np.empty(episodes, dtype=np.int32)
        data["enqueued_reqs_forward"] = np.empty(episodes, dtype=np.int32)

        # Iterate the episodes.
        for epi_idx in range(episodes):
            input_reqs = np.sum(iter_data["observation_input_requests"][epi_idx])

            # Calculate local, queued, or directly processed requests. Note that the
            # rejections must be subtracted!
            local_reqs = np.array(iter_data["action_local"][epi_idx])
            local_rejects = np.array(iter_data["local_rejects_queue_full"][epi_idx])
            real_local_reqs = np.sum(local_reqs - local_rejects)

            data["input_reqs"][epi_idx] = input_reqs
            data["enqueued_reqs_local"][epi_idx] = real_local_reqs

        # Update iteration data.
        final_data["input_reqs_avg"][iter_idx] = np.average(data["input_reqs"])
        final_data["enqueued_reqs_local_avg"][iter_idx] = np.average(
            data["enqueued_reqs_local"]
        )

    return final_data


def get_enqueued_requests_data(iter_data):
    # (experiment, iteration, metrics).
    final_data = defaultdict(lambda: defaultdict(lambda: defaultdict()))

    for exp_dir, iters in iter_data.items():
        env = base.get_env(exp_dir)
        if env.__class__ == dfaas_upperbound.SingleDFaaS:
            # The upperbound data extraction is different from the normal flow.
            final_data[exp_dir] = get_enqueued_requests_data_upperbound(iters)
            continue

        iterations = len(iters)

        # Create the portion of the dictionary for this experiment that
        # contains the average values of the metrics for each iteration.
        for agent in ["all"] + env.agents:
            final_data[exp_dir][agent]["input_reqs_avg"] = np.empty(iterations)
            final_data[exp_dir][agent]["enqueued_reqs_local_avg"] = np.empty(iterations)
            final_data[exp_dir][agent]["enqueued_reqs_forward_avg"] = np.empty(
                iterations
            )

        # For each iteration, calculate the metrics for each episode played,
        # then average the values for the number of episodes of that iteration.
        for iter_idx in range(iterations):
            episodes = iters[iter_idx]["env_runners"]["episodes_this_iter"]
            iter_data = iters[iter_idx]["env_runners"]["hist_stats"]

            # Create the data dictionary that contains the metrics for each
            # episode in this iteration.
            data = defaultdict(lambda: defaultdict())
            for agent in ["all"] + env.agents:
                data[agent]["input_reqs"] = np.zeros(episodes, dtype=np.int32)
                data[agent]["enqueued_reqs_local"] = np.zeros(episodes, dtype=np.int32)
                data[agent]["enqueued_reqs_forward"] = np.zeros(
                    episodes, dtype=np.int32
                )

            # Iterate the episodes.
            for epi_idx in range(episodes):
                for agent in env.agents:
                    input_reqs = np.sum(
                        iter_data["observation_input_requests"][epi_idx][agent]
                    )

                    # Calculate local, queued, or directly processed requests. Note that the
                    # rejections must be subtracted!
                    local_reqs = np.array(iter_data["action_local"][epi_idx][agent])
                    local_rejects = np.array(
                        iter_data["local_rejects_queue_full"][epi_idx][agent]
                    )
                    real_local_reqs = np.sum(local_reqs - local_rejects)

                    # Same for forwarded requests.
                    forwarded_reqs = np.array(
                        iter_data["action_forward"][epi_idx][agent]
                    )
                    forwarded_rejects = np.array(
                        iter_data["forward_rejects"][epi_idx][agent]
                    )
                    real_forwarded_reqs = np.sum(forwarded_reqs - forwarded_rejects)

                    data[agent]["input_reqs"][epi_idx] = input_reqs
                    data[agent]["enqueued_reqs_local"][epi_idx] = real_local_reqs
                    data[agent]["enqueued_reqs_forward"][epi_idx] = real_forwarded_reqs
                    data["all"]["input_reqs"][epi_idx] += input_reqs
                    data["all"]["enqueued_reqs_local"][epi_idx] += real_local_reqs
                    data["all"]["enqueued_reqs_forward"][epi_idx] += real_forwarded_reqs

            # Update iteration data.
            for agent in ["all"] + env.agents:
                final_data[exp_dir][agent]["input_reqs_avg"][iter_idx] = np.average(
                    data[agent]["input_reqs"]
                )
                final_data[exp_dir][agent]["enqueued_reqs_local_avg"][iter_idx] = (
                    np.average(data[agent]["enqueued_reqs_local"])
                )
                final_data[exp_dir][agent]["enqueued_reqs_forward_avg"][iter_idx] = (
                    np.average(data[agent]["enqueued_reqs_forward"])
                )

    return final_data


enqueued_reqs_data = get_enqueued_requests_data(raw_exp_data)

In [ ]:
# Postprocess additional data.
enqueued_reqs_data_avg = {"SAC": {}, "PPO": {}, "APL": {}, "Upperbound": {}}
algorithms = ["SAC", "PPO", "APL"]
for algorithm in algorithms:
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms (and environment).
        if algorithm in exp_dir.name and "SingleDFaaS" not in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
        continue

    ratios = []
    for exp in experiments:
        ratios_single_exp = (
            enqueued_reqs_data[exp]["all"]["enqueued_reqs_local_avg"]
            + enqueued_reqs_data[exp]["all"]["enqueued_reqs_forward_avg"]
        ) / enqueued_reqs_data[exp]["all"]["input_reqs_avg"]

        ratios.append(ratios_single_exp)
    ratios = np.array(ratios)

    avg = np.average(ratios, axis=0)
    std = np.std(ratios, axis=0)

    enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_avg"] = avg
    enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_std"] = std

    print(f"Algorithm {algorithm} done with {len(experiments)} experiments")


# Upperbound case is special.
def upperbound_enqueued_request():
    experiments = []
    for exp_dir in exps_dir:
        # The experiment name contains the used algorithms and environment.
        if "SingleDFaaS" in exp_dir.name:
            experiments.append(exp_dir)

    if len(experiments) == 0:
        print("WARN: Skipping upperbound, no experiments found!")
        return

    ratios = []
    for exp in experiments:
        ratios_single_exp = (
            enqueued_reqs_data[exp]["enqueued_reqs_local_avg"]
            / enqueued_reqs_data[exp]["input_reqs_avg"]
        )

        ratios.append(ratios_single_exp)
    ratios = np.array(ratios)

    avg = np.average(ratios, axis=0)
    std = np.std(ratios, axis=0)

    enqueued_reqs_data_avg["Upperbound"]["enqueued_reqs_ratios_avg"] = avg
    enqueued_reqs_data_avg["Upperbound"]["enqueued_reqs_ratios_std"] = std

    print(f"Upperbound done with {len(experiments)} experiments")


upperbound_enqueued_request()

### Average enqueued requests per episode

#### All agents

In [ ]:
def make_avg_enqueued_reqs_plot_all_agents():
    plt.close(fig="avg_enqueued_reqs_all_agents")
    fig = plt.figure(num="avg_enqueued_reqs_all_agents", layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    iteration_coords_x = np.linspace(start=1, stop=300, num=300)

    for algorithm in algorithms + ["Upperbound"]:
        if len(enqueued_reqs_data_avg[algorithm]) == 0:
            print(f"WARN: Skipping algorithm {algorithm}, no experiments found!")
            continue

        avg = enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_avg"]
        std = enqueued_reqs_data_avg[algorithm]["enqueued_reqs_ratios_std"]

        ax.plot(iteration_coords_x, avg, label=algorithm)
        ax.fill_between(iteration_coords_x, avg - std, avg + std, alpha=0.5)

    ax.set_title("Average enqueued requests per episode (all agents)")

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Show 10% ticks.
    ax.set_ylim(0, 1)

    ax.set_xlabel("Iteration")

    ax.legend()
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.


make_avg_enqueued_reqs_plot_all_agents()